In [1]:
import sqlite3
from sqlite3 import Error

import pandas as pd
from IPython.display import Image

pd.options.mode.chained_assignment = None

In [2]:
def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn

def close_connection(conn):
    try:
        if conn:
            conn.close()
            conn = None
    except Error as e:
        print(e)


def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def insert_sql_statement(insert_data, conn, table_name):
    with conn:
        cur = conn.cursor()

        empty_value_container = "?,"*len(insert_data[0])
        empty_value_container = "".join(["(",empty_value_container.strip(","),")"])

        cur.executemany(f"INSERT OR IGNORE INTO {table_name} VALUES {empty_value_container}", insert_data)
        
def read_sql_query(query, conn):
    result = pd.read_sql_query(query, conn)
    return result

In [3]:
database_filename = '/home/opc/f1_info_2021.db'
conn = create_connection(database_filename, delete_db=True)

In [4]:
def create_constructors_table():
    
    with open("/home/opc/formule1/constructors.csv","r") as f:
        data = f.read().strip().split("\n")

    data = list(map(lambda row: row.split(","), data))

    columns = data[0]

    constructor_index = columns.index("constructorId")
    name_index = columns.index("name")

    constructorId = list(map(lambda row: int(row[constructor_index]), data[1:]))
    name = list(map(lambda row: row[name_index].strip('"'), data[1:]))

    insert_data = list(zip(constructorId,name))
    
    sql_create_statement = """CREATE TABLE constructors
                        (
                            constructorId Integer not null Primary key,
                            name Text not null
                        )
                        """

    create_table(conn, sql_create_statement,drop_table_name=True)

    insert_sql_statement(insert_data, conn, "constructors")
    
def create_constructor_standings_table():

    with open("/home/opc/formule1/constructor_standings.csv","r") as f:
        data = f.read().strip().split("\n")

    data = list(map(lambda row: row.split(","), data))

    columns = data[0]

    constructorStandingsId_index = columns.index("constructorStandingsId")
    raceId_index = columns.index("raceId")
    constructorId_index = columns.index("constructorId")
    points_index = columns.index("points")
    position_index = columns.index("position")

    constructorStandingsId = list(map(lambda row: int(row[constructorStandingsId_index]), data[1:]))
    raceId = list(map(lambda row: int(row[raceId_index]), data[1:]))
    constructorId = list(map(lambda row: int(row[constructorId_index]), data[1:]))
    points = list(map(lambda row: float(row[points_index]), data[1:]))
    position = list(map(lambda row: int(row[position_index]), data[1:]))

    insert_data = list(zip(constructorStandingsId, raceId, constructorId, points, position))

    sql_create_statement = """CREATE TABLE constructor_standings
                        (
                            constructorStandingsId Integer not null Primary key,
                            raceId Integer not null,
                            constructorId not null,
                            points Integer not null,
                            position Integer not null
                        )
                        """

    create_table(conn, sql_create_statement,drop_table_name=True)

    insert_sql_statement(insert_data, conn, "constructor_standings")
    
def create_drivers_table():

    with open("/home/opc/formule1/drivers.csv","r") as f:
        data = f.read().strip().split("\n")

    data = list(map(lambda row: row.split(","), data))

    columns = data[0]

    driverId_index = columns.index("driverId")
    forename_index = columns.index("forename")
    surname_index = columns.index("surname")

    driverId = list(map(lambda row: int(row[driverId_index]), data[1:]))
    name = list(map(lambda row: " ".join([row[forename_index].strip('"'),row[surname_index].strip('"')]) , data[1:]))

    insert_data = list(zip(driverId, name))

    sql_create_statement = """CREATE TABLE drivers
                        (
                            driverId Integer not null Primary key,
                            name Text not null
                        )
                        """

    create_table(conn, sql_create_statement,drop_table_name=True)

    insert_sql_statement(insert_data, conn, "drivers")
    
def create_races_table():

    with open("/home/opc/formule1/races.csv","r") as f:
        data = f.read().strip().split("\n")

    data = list(map(lambda row: row.split(","), data))

    columns = data[0]

    raceId_index = columns.index("raceId")
    year_index = columns.index("year")
    round_index = columns.index("round")

    raceId = list(map(lambda row: int(row[raceId_index]), data[1:]))
    year = list(map(lambda row: int(row[year_index]), data[1:]))
    round = list(map(lambda row: int(row[round_index]), data[1:]))

    insert_data = list(zip(raceId, year, round))

    sql_create_statement = """CREATE TABLE races
                        (
                            raceId Integer not null Primary key,
                            year Integer not null,
                            round Integer not null
                        )
                        """

    create_table(conn, sql_create_statement, drop_table_name=True)

    insert_sql_statement(insert_data, conn, "races")
    
def create_results_table():

    with open("/home/opc/formule1/results.csv","r") as f:
        data = f.read().strip().split("\n")

    data = list(map(lambda row: row.split(","), data))

    columns = data[0]

    resultId_index = columns.index("resultId")
    raceId_index = columns.index("raceId")
    driverId_index = columns.index("driverId")
    constructorId_index = columns.index("constructorId")
    grid_index = columns.index("grid")
    position_index = columns.index("position")
    points_index = columns.index("points")
    statusId_index = columns.index("statusId")

    resultId = list(map(lambda row: int(row[resultId_index]), data[1:]))
    raceId = list(map(lambda row: int(row[raceId_index]), data[1:]))
    driverId = list(map(lambda row: int(row[driverId_index]), data[1:]))
    constructorId = list(map(lambda row: int(row[constructorId_index]), data[1:]))
    grid = list(map(lambda row: int(row[grid_index]), data[1:]))
    position = list(map(lambda row: row[position_index], data[1:]))
    points = list(map(lambda row: float(row[points_index]), data[1:]))
    statusId = list(map(lambda row: int(row[statusId_index]), data[1:]))

    position = list(map(lambda val: '1000' if val=="\\N" else val, position))
    position = list(map(lambda val: int(val), position))

    insert_data = list(zip(resultId, raceId, driverId, constructorId, grid, position, points, statusId))

    result_df = pd.DataFrame(insert_data, columns=["resultId","raceId","driverId","constructorId","grid","position","points","statusId"])

    grouped=result_df.groupby(by='raceId')['position']
    values=grouped.transform(lambda x: len(x))
    indices_to_replace=result_df[result_df.position==1000].index.tolist()
    values_to_replace=values[indices_to_replace]
    result_df['position'].iloc[indices_to_replace]=values_to_replace

    insert_data = result_df.values

    sql_create_statement = """CREATE TABLE results
                        (
                            resultId Integer not null Primary key,
                            raceId Integer not null,
                            driverId Integer not null,
                            constructorId Integer not null,
                            grid Integer not null,
                            position Integer not null,
                            points Real not null,
                            statusId Integer not null
                        )
                        """

    create_table(conn, sql_create_statement, drop_table_name=True)

    insert_sql_statement(insert_data, conn, "results")
    
def create_status_table():

    with open("/home/opc/formule1/status.csv","r") as f:
        data = f.read().strip().split("\n")

    data = list(map(lambda row: row.split(","), data))

    columns = data[0]

    statusId_index = columns.index("statusId")
    status_index = columns.index("status")

    statusId = list(map(lambda row: int(row[statusId_index]), data[1:]))
    status = list(map(lambda row: row[status_index].strip('"'), data[1:]))

    insert_data = list(zip(statusId, status))

    sql_create_statement = """CREATE TABLE status
                        (
                            statusId Integer not null Primary key,
                            status Text not null
                        )
                        """

    create_table(conn, sql_create_statement, drop_table_name=True)

    insert_sql_statement(insert_data, conn, "status")
    
def create_lap_times_table():
    
    with open("/home/opc/formule1/lap_times.csv","r") as f:
        data = f.read().strip().split("\n")

    data = list(map(lambda row: row.split(","), data))

    columns = data[0]

    raceId_index = columns.index("raceId")
    driverId_index = columns.index("driverId")
    lap_index = columns.index("lap")
    milliseconds_index = columns.index("milliseconds")

    raceId = list(map(lambda row: int(row[raceId_index]), data[1:]))
    driverId = list(map(lambda row: int(row[driverId_index]), data[1:]))
    lap = list(map(lambda row: int(row[lap_index]), data[1:]))
    milliseconds = list(map(lambda row: int(row[milliseconds_index]), data[1:]))
    lapId = list(range(1, len(raceId)+1))

    insert_data = list(zip(lapId, raceId, driverId, lap, milliseconds))

    sql_create_statement = """CREATE TABLE lap_times
                        (
                            lapId Integer not null Primary key,
                            raceId Integer not null,
                            driverId Integer not null,
                            lap Integer not null,
                            milliseconds Integer not null
                        )
                        """

    create_table(conn, sql_create_statement, drop_table_name=True)

    insert_sql_statement(insert_data, conn, "lap_times")
    
def create_pit_stops_table():
    
    with open("/home/opc/formule1/pit_stops.csv","r") as f:
        data = f.read().strip().split("\n")

    data = list(map(lambda row: row.split(","), data))

    columns = data[0]

    raceId_index = columns.index("raceId")
    driverId_index = columns.index("driverId")
    stop_index = columns.index("stop")
    lap_index = columns.index("lap")
    milliseconds_index = columns.index("milliseconds")

    raceId = list(map(lambda row: int(row[raceId_index]), data[1:]))
    driverId = list(map(lambda row: int(row[driverId_index]), data[1:]))
    stop = list(map(lambda row: int(row[stop_index]), data[1:]))
    lap = list(map(lambda row: int(row[lap_index]), data[1:]))
    milliseconds = list(map(lambda row: int(row[milliseconds_index]), data[1:]))
    pit_stop_Id = list(range(1, len(raceId)+1))

    insert_data = list(zip(pit_stop_Id, raceId, driverId, stop, lap, milliseconds))

    sql_create_statement = """CREATE TABLE pit_stops
                        (
                            pit_stop_Id Integer not null Primary key,
                            raceId Integer not null,
                            driverId Integer not null,
                            stop Integer not null,
                            lap Integer not null,
                            milliseconds Integer not null
                        )
                        """
    
    create_table(conn, sql_create_statement, drop_table_name=True)

    insert_sql_statement(insert_data, conn, "pit_stops")

In [5]:
create_constructors_table()
create_constructor_standings_table()
create_drivers_table()
create_races_table()
create_results_table()
create_status_table()
create_lap_times_table()
create_pit_stops_table()

In [6]:
# Close the DB connection
close_connection(conn)